In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mlb
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import math
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

#palette : {deep, muted, pastel, dark, bright, colorblind}
sns.set_color_codes("dark")

In [ ]:
def convert_existing_pca_rs(input_rs, cookbook):
    """ This function takes existing PCA RS and outputs the new RS format."""
    
    req_set = set()

    pca_filter = input_rs['Req Source'] == 'PCA'

    input_rs = input_rs[pca_filter].copy()
    
    for i in input_rs['Req nr']:
        #print i
        #if "H" not in i:
        req_set.add(i)
            
    #print req_set
        
    
    cb_reqs = pd.Series(cookbook['Req nr'], copy=True)
    
    #cookbook['Req nr'].replace(regex=True,inplace=True,to_replace=r'H',value=r'')
    #cookbook['Req nr'].replace(regex=True,inplace=True,to_replace=r'V',value=r'')
    #cookbook['Req nr'].replace(regex=True,inplace=True,to_replace=r'a',value=r'')
    #cookbook['Req nr'].replace(regex=True,inplace=True,to_replace=r'b',value=r'')
          
    compliancy_list = {}
    i = 0
    for x in input_rs['Req nr']:
        compliancy_list.update({x:list(input_rs.loc[input_rs['Req nr'] == x, 'Compliance Assessment'])[0]})
        i = i + 1
    
    feedback_list = {}
    i = 0
    for x in input_rs['Req nr']:
        feedback_list.update({x:list(input_rs.loc[input_rs['Req nr'] == x, 'Required additional controls/Compliance Feedback'])[0]})
        i = i + 1
    

    response_list = {}
    i = 0
    for x in input_rs['Req nr']:
        response_list.update({x:input_rs.loc[input_rs['Req nr'] == x, 'Response']})
        i = i + 1
    
    #print '~~~~~~~~~~~~~~~'
    #print response_list
        
    remref_list = {}
    i = 0
    for x in input_rs['Req nr']:
        remref_list.update({x:list(input_rs.loc[input_rs['Req nr'] == x, 'Remediation Project Ref'])[0]})
        i = i + 1
    
    res_comments_list = {}
    i = 0
    for x in input_rs['Req nr']:
        res_comments_list.update({x:list(input_rs.loc[input_rs['Req nr'] == x, 'Response Comments'])[0]})
        i = i + 1
    
    compliancy_series = []
    feedback_series = []
    response_series = []
    remref_series = []
    res_comments_series = []
    
    #print response_list

    cookbook['Compliance Assessment'] = ''
    cookbook['Required additional controls/Compliance Feedback'] = ''
    cookbook['Response'] = ''
    cookbook['Remediation Project Ref'] = ''
    cookbook['Response Comments'] = ''
    
    #print req_set

    #cookbook['Req nr'] = cb_reqs

    
    for i in range(0,107):
        if cookbook.iloc[i]['Req nr'] in req_set:
            print cookbook.iloc[i]['Req nr']
            if cookbook.iloc[i]['Req nr'] == '12H':
                print response_list[cookbook.iloc[i]['Req nr']]
                
            compliancy_series.append(compliancy_list[cookbook.iloc[i]['Req nr']])
            feedback_series.append(feedback_list[cookbook.iloc[i]['Req nr']])
            response_series.append(response_list[cookbook.iloc[i]['Req nr']])
            remref_series.append(remref_list[cookbook.iloc[i]['Req nr']])
            res_comments_series.append(res_comments_list[cookbook.iloc[i]['Req nr']])
            
            cookbook.loc[cookbook['Req nr'] == cookbook.iloc[i]['Req nr'], 'Response'] = response_list[cookbook.iloc[i]['Req nr']]
            cookbook.loc[cookbook['Req nr'] == cookbook.iloc[i]['Req nr'], 'Compliance Assessment'] = compliancy_list[cookbook.iloc[i]['Req nr']]
            cookbook.loc[cookbook['Req nr'] == cookbook.iloc[i]['Req nr'], 'Remediation Project Ref'] = remref_list[cookbook.iloc[i]['Req nr']]
            cookbook.loc[cookbook['Req nr'] == cookbook.iloc[i]['Req nr'], 'Response Comments'] = res_comments_list[cookbook.iloc[i]['Req nr']]
            cookbook.loc[cookbook['Req nr'] == cookbook.iloc[i]['Req nr'], 'Required additional controls/Compliance Feedback'] = feedback_list[cookbook.iloc[i]['Req nr']]
        else:
            compliancy_series.append(float('nan'))
            feedback_series.append(float('nan'))
            response_series.append(float('nan'))
            remref_series.append(float('nan'))
            res_comments_series.append(float('nan'))

    #print '###########################'
    #print cookbook.iloc[5]['Req nr']
    
    #print len(cb_reqs)
    #print len(response_series)
    req_list = list(cb_reqs)
    #print len(req_list)
    #print req_list
    resp_df = pd.DataFrame({'Req nr':list(cb_reqs), 
                            'Response':response_series, 
                            'Compliance Assessment':compliancy_series,
                            'Remediation Project Ref':remref_series,
                            'Response Comments':res_comments_series,
                            'Required additional controls/Compliance Feedback':feedback_series})
    #print resp_df
    #print '###########################'
    
    
    cookbook.sort_values(by=['Compliance Assessment','Req nr'],ascending=True, inplace=True)
    
    cookbook.rename(columns={'Cookbook RS Statements':'GDPR Requirement Specification',
                             'Privacy Requirement':'Requirement',
                             'People+Process Applicable':'People/Process Applicability',
                             'Technology  (& Design) Applicable':'Technology/Design Applicability'}, inplace=True)
    
    comliancy_filter = cookbook['Compliance Assessment'] != float('nan')
        
    return cookbook[comliancy_filter]

In [ ]:
def convert_existing_sca_rs(input_rs, cookbook):
    """ This function takes existing SCA RS and outputs the new RS format."""
    
    req_set = set()

    sca_filter = input_rs['Req Source'] == 'SCA'

    input_rs = input_rs[sca_filter].copy()
    
    for i in input_rs['Req nr']:
        #print i
        req_set.add(i)
        
    
    cb_reqs = pd.Series(cookbook['RS Reference Number'], copy=True)
    
    compliancy_list = {}
    i = 0
    for x in input_rs['Req nr']:
        compliancy_list.update({x:input_rs.iloc[i]['Compliance Assessment']})
        i = i + 1
    
    feedback_list = {}
    i = 0
    for x in input_rs['Req nr']:
        feedback_list.update({x:input_rs.iloc[i]['Required additional controls/Compliance Feedback']})
        i = i + 1
    

    response_list = {}
    i = 0
    for x in input_rs['Req nr']:
        response_list.update({x:input_rs.iloc[i]['Response']})
        i = i + 1
        
    remref_list = {}
    i = 0
    for x in input_rs['Req nr']:
        remref_list.update({x:input_rs.iloc[i]['Remediation Project Ref']})
        i = i + 1
    
    res_comments_list = {}
    i = 0
    for x in input_rs['Req nr']:
        res_comments_list.update({x:input_rs.iloc[i]['Response Comments']})
        i = i + 1
    
    compliancy_series = []
    feedback_series = []
    response_series = []
    remref_series = []
    res_comments_series = []
    
    for i in range(0,15):
        if cookbook.iloc[i]['RS Reference Number'][10:-2] in req_set:
            compliancy_series.append(compliancy_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
            feedback_series.append(feedback_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
            response_series.append(response_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
            remref_series.append(remref_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
            res_comments_series.append(res_comments_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
        else:
            compliancy_series.append(float('nan'))
            feedback_series.append(float('nan'))
            response_series.append(float('nan'))
            remref_series.append(float('nan'))
            res_comments_series.append(float('nan'))
            
    
    resp_df = pd.DataFrame({'Req nr':list(cb_reqs), 
                            'Response':response_series, 
                            'Compliance Assessment':compliancy_series,
                            'Remediation Project Ref':remref_series,
                            'Response Comments':res_comments_series,
                            'Required additional controls/Compliance Feedback':feedback_series})
    #print resp_df
    #print '###########################'
    
    cookbook['Response'] = resp_df['Response']
    cookbook['Compliance Assessment'] = resp_df['Compliance Assessment']
    cookbook['Remediation Project Ref'] = resp_df['Remediation Project Ref']
    cookbook['Response Comments'] = resp_df['Response Comments']
    cookbook['Required additional controls/Compliance Feedback'] = resp_df['Required additional controls/Compliance Feedback']
    
    cookbook.sort_values(by=['Compliance Assessment'],ascending=True, inplace=True)
    
    cookbook['Req nr'] = cb_reqs
    
    cookbook.rename(columns={'Cookbook RS Statements':'GDPR Requirement Specification',
                             'Privacy Requirement':'Requirement',
                             'People+Process Applicable':'People/Process Applicability',
                             'Technology  (& Design) Applicable':'Technology/Design Applicability'}, inplace=True)
    
    comliancy_filter = cookbook['Compliance Assessment'] != float('nan')
        
    return cookbook[comliancy_filter]

In [ ]:
os.chdir('/home/andy/dev/notebooks/Transfer-RS-Feedback/RS_FEEDBACK/IoT')

#OUTPUT_BASE_PATH = '/home/andy/dev/notebooks/Transfer-RS-Feedback/output_rs/'
OUTPUT_BASE_PATH = '/home/andy/dev/GDPR/converted_rs/'
OUT_FILENAME = 'outfile.xlsx'

COOKBOOK_FILE = '/home/andy/dev/GDPR/inputdata/PCA SCA Cookbook Mapping_3.4.xlsx'

cookbook = pd.read_excel(COOKBOOK_FILE, 
                         sheet_name='PCA',
                         skiprows=1)

cookbook_sca = pd.read_excel(COOKBOOK_FILE, 
                         sheet_name='SCA',
                         skiprows=1)

first = True
new_rs_sheets = {}

for filename in os.listdir(os.getcwd()):
    print filename
    output_writer = pd.ExcelWriter(OUTPUT_BASE_PATH + 'CONVERTED-PCA-' + filename[:14] + '.xlsx')

    df = pd.read_excel(filename, sheet_name=None)
    
    sheet_names = df.keys()
    sheets = sheet_names[4:-1]
    #print sheets

    for sh in sheets:
        print sh
        if sh == u'PII Data Record': # this is a bug? Don't know why I need this.  For some reason the last sheet comes through.
            continue
        #new_rs_sheets.update({filename: dict({sh: convert_rs(df[sh], cookbook)})})
        
        print df[sh][['Req nr','Response']]
        out_df = convert_existing_pca_rs(df[sh], cookbook)
        
        out_df.to_excel(output_writer,sh)
        
    output_writer.save()

In [ ]:
for filename in os.listdir(os.getcwd()):
    output_writer = pd.ExcelWriter(OUTPUT_BASE_PATH + 'CONVERTED-SCA-' + filename[:14] + '.xlsx')

    df = pd.read_excel(filename, sheet_name=None)
    
    sheet_names = df.keys()
    sheets = sheet_names[4:-1]
    #print sheets

    for sh in sheets:
        #print sh
        if sh == u'PII Data Record': # this is a bug? Don't know why I need this.  For some reason the last sheet comes through.
            continue
        #new_rs_sheets.update({filename: dict({sh: convert_rs(df[sh], cookbook)})})
        out_df = convert_existing_sca_rs(df[sh], cookbook_sca)
        out_df.to_excel(output_writer,sh)
        
    output_writer.save()

In [ ]:
out_df.loc[out_df['Req nr'] == 'EPSXXX-SCA9.2.2-Y', 'Response'] = 'AJB'

In [ ]:
list(out_df.loc[out_df['Req nr'] == 'EPSXXX-SCA9.2.2-Y', 'Response'])[0]

In [ ]:
nan = ''

In [ ]:
test_list = [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, u'Rejected', nan, u'Rejected', nan, nan, nan, nan, nan, nan, nan, u'Rejected', nan, nan, u'Rejected', nan, u'Acknowledged', nan, nan, nan, nan, nan, nan, nan, nan, u'Acknowledged', nan, nan, nan, u'Rejected', nan, u'Acknowledged', nan, nan, nan, nan]

In [ ]:
len(test_list)